In [ ]:
import scipp as sc

## Loading dataset

> Loader is not part of ``essimaging`` since McStas dataset format is not stabilized yet.

In [ ]:
import scippnexus as snx
from typing import cast, NewType
import numpy as np
from ess.reduce.nexus.types import FilePath


_DataPath = NewType('_DataPath', str)
_DefaultDataPath = _DataPath(
    "entry1/data/transmission_event_signal_dat_list_p_t_x_y_z_vx_vy_vz/events"
)
_FileLock = NewType('_FileLock', bool)
"""Lock the file to prevent concurrent access."""
_DefaultFileLock = _FileLock(True)
OdinSimulationRawData = NewType('OdinSimulationRawData', sc.DataArray)
ProbabilityToCountsScaleFactor = NewType('ProbabilityToCountsScaleFactor', sc.Variable)
"""Translate the probability to counts."""
DefaultProbabilityToCountsScaleFactor = ProbabilityToCountsScaleFactor(sc.scalar(1_000, unit='dimensionless'))



def load_odin_simulation_data(
    file_path: FilePath,
    _data_path: _DataPath = _DefaultDataPath,
    _file_lock: _FileLock = _DefaultFileLock,
    probability_scale_factor: ProbabilityToCountsScaleFactor = DefaultProbabilityToCountsScaleFactor,
) -> OdinSimulationRawData:
    with snx.File(file_path, "r", locking=_file_lock) as f:
        data = f[_data_path][()].rename_dims({'dim_0': 'event'})
        probabilities = cast(sc.Variable, data['dim_1', 0].copy())
        probabilities.unit = 'dimensionless'
        time_of_arrival = data['dim_1', 1]
        positions = data['dim_1', 2:5]
        velocities = data['dim_1', 5:8]
        counts = ((probabilities / probabilities.max()) * probability_scale_factor).astype(int)
        da = sc.DataArray(
            data=counts,
            coords={
                'time_of_arrival': cast(sc.Variable, time_of_arrival),
                'x': cast(sc.Variable, positions['dim_1', 0]),
                'y': cast(sc.Variable, positions['dim_1', 1]),
                'z': cast(sc.Variable, positions['dim_1', 2]),
                'position': sc.vectors(
                    dims=['event'], values=cast(np.ndarray, positions.values)
                ),
                'vx': cast(sc.Variable, velocities['dim_1', 0]),
                'vy': cast(sc.Variable, velocities['dim_1', 1]),
                'vz': cast(sc.Variable, velocities['dim_1', 2]),
                'velocity': sc.vectors(
                    dims=['event'], values=cast(np.ndarray, velocities.values)
                ),
            },
        )
        return OdinSimulationRawData(da)



file_path = FilePath("task_bragg_baseline_open/data/mccode.h5")
load_odin_simulation_data(file_path)

## Choppers

> Choppers can be retrieved from nexus file automatically. <br>
> We are hardcoding them for simulation data reduction. <br>
> We may automate this once McStas nexus format is stabilized.

In [ ]:
from scippneutron.chopper import DiskChopper

# Collect choppers
## WFM choppers
wfm_frequency = sc.scalar(value=56.0, unit='Hz')
beam_angle = sc.scalar(value=0.0, unit='deg')

WFMC_1 = DiskChopper(
    axle_position=sc.vector(value=(0.026000, 0.000000, 6.850000), unit='m'),
    frequency=wfm_frequency,
    beam_angle=beam_angle,
    phase=sc.scalar(value=93.244, unit='deg'),
    slit_begin=sc.array(
        dims=['slit'],
        values=[-1.9419, 49.5756, 98.9315, 146.2165, 191.5176, 234.9179],
        unit='deg',
    ),
    slit_end=sc.array(
        dims=['slit'],
        values=[1.9419, 55.7157, 107.2332, 156.5891, 203.8741, 249.1752],
        unit='deg',
    ),
)

WFMC_2 = DiskChopper(
    axle_position=sc.vector(value=(0.026000, 0.000000, 7.150000), unit='m'),
    frequency=wfm_frequency,
    beam_angle=beam_angle,
    phase=sc.scalar(value=152.029879, unit='deg'),
    slit_begin=sc.array(
        dims=['slit'],
        values=[-1.9419, 51.8318, 103.3493, 152.7052, 199.9903, 245.2914],
        unit='deg',
    ),
    slit_end=sc.array(
        dims=['slit'],
        values=[1.9419, 57.9719, 111.6510, 163.0778, 212.3468, 259.5486],
        unit='deg',
    ),
)

In [ ]:
WFMC_1

In [ ]:
WFMC_2

In [ ]:
## FOC choppers
foc_frequency = sc.scalar(value=42.0, unit='Hz')
F01 = DiskChopper(
    axle_position=sc.vector(value=(0.026000, 0.000000, 8.400000), unit='m'),
    frequency=foc_frequency,
    beam_angle=beam_angle,
    phase=sc.scalar(value=81.303297, unit='deg'),
    slit_begin=sc.array(
        dims=['slit'],
        values=[-5.1362, 42.5536, 88.2425, 132.0144, 173.9497, 216.7867],
        unit='deg',
    ),
    slit_end=sc.array(
        dims=['slit'],
        values=[5.1362, 54.2095, 101.2237, 146.2653, 189.417, 230.7582],
        unit='deg',
    ),
)

F02 = DiskChopper(
    axle_position=sc.vector(value=(0.026000, 0.000000, 12.200000), unit='m'),
    frequency=foc_frequency,
    beam_angle=beam_angle,
    phase=sc.scalar(value=107.013442, unit='deg'),
    slit_begin=sc.array(
        dims=['slit'],
        values=[-16.3227, 53.7401, 120.8633, 185.1701, 246.7787, 307.0165],
        unit='deg',
    ),
    slit_end=sc.array(
        dims=['slit'],
        values=[16.3227, 86.8303, 154.3794, 218.7551, 280.7508, 340.3188],
        unit='deg',
    ),
)

F03 = DiskChopper(
    axle_position=sc.vector(value=(0.026000, 0.000000, 17.000000), unit='m'),
    frequency=foc_frequency,
    beam_angle=beam_angle,
    phase=sc.scalar(value=158.294923, unit='deg'),
    slit_begin=sc.array(
        dims=['slit'],
        values=[-20.302, 45.247, 108.0457, 168.2095, 225.8489, 282.2199],
        unit='deg',
    ),
    slit_end=sc.array(
        dims=['slit'],
        values=[20.302, 85.357, 147.6824, 207.3927, 264.5977, 319.4024],
        unit='deg',
    ),
)

F04 = DiskChopper(
    axle_position=sc.vector(value=(0.026000, 0.000000, 23.690000), unit='m'),
    frequency=foc_frequency,
    beam_angle=beam_angle,
    phase=sc.scalar(value=61.584, unit='deg'),
    slit_begin=sc.array(
        dims=['slit'],
        values=[-16.7157, 29.1882, 73.1661, 115.2988, 155.6636, 195.5254],
        unit='deg',
    ),
    slit_end=sc.array(
        dims=['slit'],
        values=[16.7157, 61.8217, 105.0352, 146.4355, 186.0987, 224.0978],
        unit='deg',
    ),
)

F05 = DiskChopper(
    axle_position=sc.vector(value=(0.026000, 0.000000, 33.000000), unit='m'),
    frequency=foc_frequency,
    beam_angle=beam_angle,
    phase=sc.scalar(value=145.973844, unit='deg'),
    slit_begin=sc.array(
        dims=['slit'],
        values=[-25.8514, 38.3239, 99.8064, 160.1254, 217.4321, 272.5426],
        unit='deg',
    ),
    slit_end=sc.array(
        dims=['slit'],
        values=[25.8514, 88.4621, 147.4729, 204.0245, 257.7603, 313.7139],
        unit='deg',
    ),
)

F05

In [ ]:
## BP choppers
bp_frequency = sc.scalar(value=7.0, unit='Hz')
BP01 = DiskChopper(
    axle_position=sc.vector(value=(0.026000, 0.000000, 8.450000), unit='m'),
    frequency=bp_frequency,
    beam_angle=beam_angle,
    phase=sc.scalar(value=31.079597, unit='deg'),
    slit_begin=sc.array(dims=['slit'], values=[-23.6029], unit='deg'),
    slit_end=sc.array(dims=['slit'], values=[23.6029], unit='deg'),
    radius=sc.scalar(value=0.5, unit='m'),
    slit_height=sc.scalar(value=0.075000, unit='m'),
)

BP02 = DiskChopper(
    axle_position=sc.vector(value=(0.026000, 0.000000, 12.250000), unit='m'),
    frequency=bp_frequency,
    beam_angle=beam_angle,
    phase=sc.scalar(value=44.223912, unit='deg'),
    slit_begin=sc.array(dims=['slit'], values=[-34.4663], unit='deg'),
    slit_end=sc.array(dims=['slit'], values=[34.4663], unit='deg'),
    radius=sc.scalar(value=0.5, unit='m'),
    slit_height=sc.scalar(value=0.080000, unit='m'),
)

BP02

In [ ]:
# T0 chopppers
t0_frequency = sc.scalar(value=14.0, unit='Hz')

TAlpha = DiskChopper(
    axle_position=sc.vector(value=(0.026000, 0.000000, 13.500000), unit='m'),
    frequency=t0_frequency,
    beam_angle=beam_angle,
    phase=sc.scalar(value=179.672400, unit='deg'),
    slit_begin=sc.array(dims=['slit'], values=[-167.8986], unit='deg'),
    slit_end=sc.array(dims=['slit'], values=[167.8986], unit='deg'),
    radius=sc.scalar(value=0.3, unit='m'),
    slit_height=sc.scalar(value=0.075000, unit='m'),
)

TBeta = DiskChopper(
    axle_position=sc.vector(value=(0.000000, 0.000000, 0.200000), unit='m'),
    frequency=t0_frequency,
    beam_angle=beam_angle,
    phase=sc.scalar(value=179.672, unit='deg'),
    slit_begin=sc.array(dims=['slit'], values=[-167.8986], unit='deg'),
    slit_end=sc.array(dims=['slit'], values=[167.8986], unit='deg'),
    radius=sc.scalar(value=0.3, unit='m'),
    slit_height=sc.scalar(value=0.075000, unit='m'),
)
TBeta